In [180]:
# TODO: перевести все файлы либо в .soft, либо в .csv

from GEOparse import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import warnings
from functools import cmp_to_key

In [185]:
phenotype_data = pd.DataFrame(pd.read_table("./data/segregant_phenotype_annotation.tab"))
last_seg_col = phenotype_data.columns.tolist().index("17_1_a")
# strains = phenotype_data.columns.tolist()[1:last_seg_col+1]


def strain_to_list(item):
    l = item.split('_')
    return [int(l[0]), int(l[1]), ord(l[2])]

strains = sorted(
    phenotype_data.columns.tolist()[1:last_seg_col+1],
    key=strain_to_list
)

print(strains)

['1_1_d', '1_2_d', '1_3_d', '1_4_d', '1_5_c', '2_1_d', '2_2_d', '2_3_d', '2_4_a', '2_5_d', '2_6_d', '2_7_a', '2_7_b', '2_7_c', '2_7_d', '3_1_d', '3_2_d', '3_3_d', '3_4_d', '3_5_d', '4_1_c', '4_2_a', '4_3_d', '4_4_d', '5_1_d', '5_2_d', '5_3_d', '5_4_d', '5_5_d', '6_1_d', '6_2_b', '6_3_c', '6_4_d', '6_5_d', '6_6_d', '6_7_d', '7_1_d', '7_2_c', '7_3_d', '7_4_c', '7_5_d', '7_6_c', '7_7_c', '7_8_d', '8_1_a', '8_2_d', '8_3_a', '8_4_c', '8_5_b', '8_6_c', '8_7_b', '9_1_d', '9_2_d', '9_3_d', '9_4_d', '9_5_d', '9_6_d', '9_7_d', '10_1_c', '10_2_d', '10_3_c', '10_4_d', '11_1_a', '11_2_d', '11_3_b', '12_1_d', '12_2_b', '13_1_a', '13_2_c', '13_3_b', '13_4_a', '13_5_c', '14_1_b', '14_2_c', '14_3_d', '14_4_a', '14_5_b', '14_6_d', '14_7_c', '15_2_d', '15_3_b', '15_4_d', '15_5_b', '15_6_c', '16_1_d', '17_1_a', '17_2_d', '17_4_a', '17_5_b', '18_1_d', '18_2_d', '18_3_d', '18_4_c', '18_6_d', '19_1_c', '19_2_c', '19_3_c', '19_4_b', '19_5_b', '20_1_d', '20_2_d', '20_3_c', '20_4_c', '20_5_d', '21_1_d', '21_2_d

In [191]:
%%javascript
# suppressing annoying "Unknown subset type" warnings
{$("div.output_stderr").toggle();}
expression_data = GEOparse.parse_GDS(
    filepath="./data/GDS1115.soft"
)

<IPython.core.display.Javascript object>

In [199]:
GSMs_list = expression_data.columns.tolist()
print("{}: {}".format(len(GSMs_list), GSMs_list))
print(GEOparse.parse_GDS(
    filepath="./data/GDS1115.soft"
).database)

133: ['ID_REF', 'IDENTIFIER', 'GSM35588', 'GSM35590', 'GSM35592', 'GSM35594', 'GSM35596', 'GSM35598', 'GSM35600', 'GSM35602', 'GSM35604', 'GSM35606', 'GSM35608', 'GSM35610', 'GSM35612', 'GSM35614', 'GSM35616', 'GSM35618', 'GSM35620', 'GSM35622', 'GSM35624', 'GSM35626', 'GSM35628', 'GSM35630', 'GSM35632', 'GSM35634', 'GSM35636', 'GSM35638', 'GSM35640', 'GSM61980', 'GSM61982', 'GSM61984', 'GSM61986', 'GSM61988', 'GSM61990', 'GSM61992', 'GSM61994', 'GSM61996', 'GSM61998', 'GSM62000', 'GSM62002', 'GSM62004', 'GSM62006', 'GSM62008', 'GSM62010', 'GSM62012', 'GSM62014', 'GSM62016', 'GSM62018', 'GSM62020', 'GSM62022', 'GSM62024', 'GSM62026', 'GSM62028', 'GSM62030', 'GSM62032', 'GSM62034', 'GSM62036', 'GSM62038', 'GSM62040', 'GSM62042', 'GSM62044', 'GSM62046', 'GSM62048', 'GSM62050', 'GSM62052', 'GSM62054', 'GSM62056', 'GSM62058', 'GSM62060', 'GSM62062', 'GSM62064', 'GSM62066', 'GSM62068', 'GSM62070', 'GSM62072', 'GSM62074', 'GSM62076', 'GSM62078', 'GSM62080', 'GSM62082', 'GSM62084', 'GSM62086'

 - DATABASE : Geo
 - DATASET : GDS1115
 - SUBSET : GDS1115_1
 - SUBSET : GDS1115_2
 - SUBSET : GDS1115_3
 - DATASET : GDS1115
Unknown subset type: strain for subset GDS1115_1
Unknown subset type: strain for subset GDS1115_1
Unknown subset type: strain for subset GDS1115_1
Unknown subset type: strain for subset GDS1115_1
Unknown subset type: strain for subset GDS1115_1
Unknown subset type: strain for subset GDS1115_1
Unknown subset type: strain for subset GDS1115_3
Unknown subset type: strain for subset GDS1115_3
Unknown subset type: strain for subset GDS1115_3
Unknown subset type: strain for subset GDS1115_3
Unknown subset type: strain for subset GDS1115_3
Unknown subset type: strain for subset GDS1115_3
Unknown subset type: strain for subset GDS1115_3
Unknown subset type: strain for subset GDS1115_3
Unknown subset type: strain for subset GDS1115_3
Unknown subset type: strain for subset GDS1115_3
Unknown subset type: strain for subset GDS1115_3
Unknown subset type: strain for subset GD

<DATABASE: Geo>


In [160]:
print("{}: {}".format(len(strains), strains))

In [122]:
print(phenotype_data.columns.tolist())

['RQTL_name', '2_7_a', '2_7_b', '2_7_c', '2_7_d', '21_5_c', '22_1_d', '22_2_d', '19_2_c', '19_3_c', '19_4_b', '19_5_b', '20_1_d', '20_2_d', '20_3_c', '20_4_c', '20_5_d', '21_2_d', '22_3_b', '22_4_d', '22_5_d', '23_3_d', '23_4_d', '23_5_d', '24_1_d', '25_1_d', '25_3_d', '25_4_d', '26_1_d', '26_3_d', '10_1_c', '10_2_d', '10_3_c', '10_4_d', '11_1_a', '11_2_d', '11_3_b', '12_1_d', '12_2_b', '13_1_a', '13_2_c', '13_3_b', '13_4_a', '13_5_c', '14_1_b', '14_2_c', '14_3_d', '21_4_d', '14_4_a', '14_5_b', '14_6_d', '14_7_c', '15_2_d', '15_3_b', '15_4_d', '15_5_b', '16_1_d', '17_2_d', '17_4_a', '17_5_b', '18_1_d', '18_2_d', '18_3_d', '18_4_c', '18_6_d', '1_1_d', '1_2_d', '1_3_d', '1_4_d', '1_5_c', '2_1_d', '2_2_d', '2_3_d', '2_4_a', '2_5_d', '2_6_d', '3_1_d', '3_2_d', '3_3_d', '3_4_d', '3_5_d', '4_1_c', '4_2_a', '4_3_d', '4_4_d', '5_1_d', '5_2_d', '5_3_d', '5_4_d', '5_5_d', '6_1_d', '6_2_b', '6_3_c', '6_4_d', '6_5_d', '6_6_d', '6_7_d', '7_1_d', '7_2_c', '7_3_d', '7_4_c', '7_5_d', '7_6_c', '7_7_c',